In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import os 
os.chdir("/content/drive/My Drive/fast.ai v3/section 2/cust_dl_lib")

!git config --global user.email "rajp152000@gmail.com"
!git config --global user.name "rajp152k"
!pip install fire
import fire
datasets_path = "/content/drive/My Drive/fast.ai v3/datasets"

In [16]:
!git pull

Already up to date.


In [0]:
#export 
from exps.nb_03 import * 

combining the training and validation dataloaders in a single package

## Databunch

In [0]:
x_train,y_train,x_valid,y_valid = get_data()
train_ds,valid_ds = Dataset(x_train,y_train),Dataset(x_valid,y_valid)
nh= 50;bs = 64
c = 10
loss_func = F.cross_entropy

passing arguments in a compacted way by the means of a learner

read up on pythonic property objects before proceeding

In [0]:
#export 
class DataBunch():
    def __init__(self,train_dl,valid_dl,c=None):
        self.train_dl = train_dl
        self.valid_dl = valid_dl
        self.c = c
    
    @property
    def train_ds(self): return self.train_dl.dataset

    @property
    def valid_ds(self): return self.valid_dl.dataset

In [0]:
train_dl,valid_dl = get_dls(train_ds,valid_ds,bs)
data = DataBunch(train_dl,valid_dl,c)

In [21]:
data,data.train_dl,data.train_ds

(<__main__.DataBunch at 0x7f02e0333ac8>,
 <exps.nb_03.Dataset at 0x7f0338026e10>)

In [22]:
help(data)

Help on DataBunch in module __main__ object:

class DataBunch(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, train_dl, valid_dl, c=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  train_ds
 |  
 |  valid_ds



In [0]:
#export 
def get_model(data,lr=0.5,nh=50):
    m = data.train_ds.x.shape[1]
    model = nn.Sequential(nn.Linear(m,nh),
                          nn.ReLU(),
                          nn.Linear(nh,data.c))
    return model,optim.SGD(model.parameters(),lr=lr)

class Learner():
    def __init__(self,model,opt,loss_func,data):
        self.model = model
        self.opt = opt
        self.loss_func = loss_func
        self.data = data

In [0]:
learn = Learner(*get_model(data),loss_func,data)

how it looks right now:

In [0]:
def fit(epochs,learn):
    for epoch in range(epochs):
        learn.model.train()
        for xb,yb in learn.data.train_dl:
            preds = learn.model(xb)
            loss = learn.loss_func(preds,yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()

        learn.model.eval()
        with torch.no_grad():
            loss_sum,acc_sum=0.,0.
            for xb,yb in learn.data.valid_dl:
                preds=learn.model(xb)
                loss_sum += learn.loss_func(preds,yb)
                acc_sum += accuracy(preds,yb)
            
        nv = len(learn.data.valid_dl)
        print(f'epoch : {epoch} | loss : {loss_sum/nv}| accuracy : {acc_sum/nv}')
    return loss_sum/nv,acc_sum/nv


      

In [26]:
_,_ = fit(1,learn)

epoch : 0 | loss : 0.23337627947330475| accuracy : 0.9263251423835754


# Incorporating Callbacks

#CallbackHandler

the rudimentary functionality :
lots of code smells: cb occuring as a parameter in many attribute setters: refactoring as a single class later on

In [0]:
def one_batch(xb, yb, cb):
    if not cb.begin_batch(xb,yb): return
    loss = cb.learn.loss_func(cb.learn.model(xb), yb)
    if not cb.after_loss(loss): return
    loss.backward()
    if cb.after_backward(): cb.learn.opt.step()
    if cb.after_step(): cb.learn.opt.zero_grad()

def all_batches(dl, cb):
    for xb,yb in dl:
        one_batch(xb, yb, cb)
        if cb.do_stop(): return

def fit(epochs, learn, cb):
    if not cb.begin_fit(learn): return
    for epoch in range(epochs):
        if not cb.begin_epoch(epoch): continue
        all_batches(learn.data.train_dl, cb)
        
        if cb.begin_validate():
            with torch.no_grad(): all_batches(learn.data.valid_dl, cb)
        if cb.do_stop() or not cb.after_epoch(): break
    cb.after_fit()

In [0]:
class Callback():
    def begin_fit(self, learn):
        self.learn = learn
        return True
    def after_fit(self): return True
    def begin_epoch(self, epoch):
        self.epoch=epoch
        return True
    def begin_validate(self): return True
    def after_epoch(self): return True
    def begin_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        return True
    def after_loss(self, loss):
        self.loss = loss
        return True
    def after_backward(self): return True
    def after_step(self): return True

In [0]:
class CallbackHandler():
    def __init__(self,cbs=None):
        self.cbs = cbs if cbs else []

    def begin_fit(self, learn):
        self.learn,self.in_train = learn,True
        learn.stop = False
        res = True
        for cb in self.cbs: res = res and cb.begin_fit(learn)
        return res

    def after_fit(self):
        res = not self.in_train
        for cb in self.cbs: res = res and cb.after_fit()
        return res
    
    def begin_epoch(self, epoch):
        self.learn.model.train()
        self.in_train=True
        res = True
        for cb in self.cbs: res = res and cb.begin_epoch(epoch)
        return res

    def begin_validate(self):
        self.learn.model.eval()
        self.in_train=False
        res = True
        for cb in self.cbs: res = res and cb.begin_validate()
        return res

    def after_epoch(self):
        res = True
        for cb in self.cbs: res = res and cb.after_epoch()
        return res
    
    def begin_batch(self, xb, yb):
        res = True
        for cb in self.cbs: res = res and cb.begin_batch(xb, yb)
        return res

    def after_loss(self, loss):
        res = self.in_train
        for cb in self.cbs: res = res and cb.after_loss(loss)
        return res

    def after_backward(self):
        res = True
        for cb in self.cbs: res = res and cb.after_backward()
        return res

    def after_step(self):
        res = True
        for cb in self.cbs: res = res and cb.after_step()
        return res
    
    def do_stop(self):
        try:     return self.learn.stop
        finally: self.learn.stop = False

## creating a Runner

In [0]:
#export 
import re

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')

def camel2snake(name):
    s1 = re.sub(_camel_re1,r'\1_\2',name)
    return re.sub(_camel_re2,r'\1_\2',s1).lower()

class Callback():
    _order=0
    def set_runner(self,run): self.run=run
    def __getattr__(self,k): return getattr(self.run,k)
    @property
    def name(self):
        name = re.sub(r'Callback$','',self.__class__.__name__)
        return camel2snake(name or 'callback')
    

Callback is a parent class for the ones written below

the following one switches models state between training and validation also keeping a count of the iterations(including the fraction) 

In [0]:
#export
class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0
    
    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter   += 1
        
    def begin_epoch(self):
        self.run.n_epochs=self.epoch
        self.model.train()
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False
    

In [0]:
class TestCallback(Callback):
    def after_step(self):
        if self.train_eval.n_iters>=10: return True

In [33]:
cbname=  'TrainEvalCallback'
camel2snake(cbname)

'train_eval_callback'

In [35]:
TrainEvalCallback().name

'train_eval'

In [0]:
#export 
from typing import *

def listify(o):
    if o is None:return []
    if isinstance(o,list):return o
    if isinstance(o,str):return [o]
    if isinstance(o,Iterable):return list(o)
    return [o]

In [0]:
#export 
class Runner():
    def __init__(self,cbs=None,cb_funcs=None):
        cbs = listify(cbs)
        for cbf in listify(cb_funcs):
            cb = cbf()
            setattr(self,cb.name,cb)
            cbs.append(cb)
        self.stop,self.cbs = False,[TrainEvalCallback()]+cbs

    @property
    def opt(self) : return self.learn.opt
    @property
    def model(self) : return self.learn.model
    @property
    def loss_func(self) : return self.learn.loss_func
    @property
    def data(self) :return self.learn.data
    
    def one_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        if self('begin_batch'): return
        self.pred = self.model(self.xb)
        if self('after_pred'): return
        self.loss = self.loss_func(self.pred, self.yb)
        if self('after_loss') or not self.in_train: return
        self.loss.backward()
        if self('after_backward'): return
        self.opt.step()
        if self('after_step'): return
        self.opt.zero_grad()

    def all_batches(self, dl):
        self.iters = len(dl)
        for xb,yb in dl:
            if self.stop: break
            self.one_batch(xb, yb)
            self('after_batch')
        self.stop=False

    def fit(self, epochs, learn):
        self.epochs,self.learn = epochs,learn

        try:
            for cb in self.cbs: cb.set_runner(self)
            if self('begin_fit'): return
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.all_batches(self.data.train_dl)

                with torch.no_grad(): 
                    if not self('begin_validate'): self.all_batches(self.data.valid_dl)
                if self('after_epoch'): break
            
        finally:
            self('after_fit')
            self.learn = None

    def __call__(self, cb_name):
        for cb in sorted(self.cbs, key=lambda x: x._order):
            f = getattr(cb, cb_name, None)
            if f and f(): return True
        return False

metrics handler

In [0]:
#export
class AvgStats():
    def __init__(self, metrics, in_train): self.metrics,self.in_train = listify(metrics),in_train
    
    def reset(self):
        self.tot_loss,self.count = 0.,0
        self.tot_mets = [0.] * len(self.metrics)
        
    @property
    def all_stats(self): return [self.tot_loss.item()] + self.tot_mets
    @property
    def avg_stats(self): return [o/self.count for o in self.all_stats]
    
    def __repr__(self):
        if not self.count: return ""
        return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"

    def accumulate(self, run):
        bn = run.xb.shape[0]
        self.tot_loss += run.loss * bn
        self.count += bn
        for i,m in enumerate(self.metrics):
            self.tot_mets[i] += m(run.pred, run.yb) * bn

class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats,self.valid_stats = AvgStats(metrics,True),AvgStats(metrics,False)
        
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
        
    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)
    
    def after_epoch(self):
        print(self.train_stats)
        print(self.valid_stats)

Testing functionality

In [0]:
learn = Learner(*get_model(data),loss_func,data)

In [0]:
stats = AvgStatsCallback([accuracy])
run = Runner(cbs=stats)

In [59]:
run.fit(2,learn)

train: [0.30163470703125, tensor(0.9074)]
valid: [0.1544182861328125, tensor(0.9558)]
train: [0.140667470703125, tensor(0.9581)]
valid: [0.1183848876953125, tensor(0.9667)]


In [60]:
loss,acc = stats.valid_stats.avg_stats
assert acc>0.9
loss,acc

(0.1183848876953125, tensor(0.9667))

In [0]:
#export 
from functools import partial

using partial functions

In [0]:
acc_cbf = partial(AvgStatsCallback,accuracy)

In [0]:
run = Runner(cb_funcs=acc_cbf)

In [65]:
run.fit(1,learn)

train: [0.10653248046875, tensor(0.9677)]
valid: [0.2008005126953125, tensor(0.9391)]


In [66]:
run.avg_stats.valid_stats.avg_stats

[0.2008005126953125, tensor(0.9391)]

gained callback funcionality

# Exporting

In [0]:
!python notebook2script.py 04_call